# Deep Learning
Olvasd el az [elméleti bevezetőt](http://inf.u-szeged.hu/~rfarkas/deep_learning.html).

### Futtatás GPU-n

A mély neurális hálók tanítása nagyon számításigényes, viszont visszavezetve mátrixműveletekre nagyon jól párhuzamosítható GPU-n. Érdemes a Google Colab-ban is átváltani GPU-ra. Ezt az Edit>Notebook settings menüben tehetjük meg GPU-t választva hardveres gyorsításra. Ha CPU-ról átvátunk GPU-ra akkor újra kell futtatni a teljes notebookot!

A Cuda egy alacsony szintű szoftverréteg mátrixműveletek GPU-n való nagyon hatékony megvalósítására. E fölé épülnek a deep learning keretrendszerek. A két legelterjedtebb keretrendszer a [PyTorch](https://pytorch.org/) és a [Tensorflow](https://www.tensorflow.org/).

In [ ]:
### PyTorch deep learning keretrendszert használjuk: https://pytorch.org
import torch  

In [ ]:
### Futtatási környezet előkészítése

# Cuda inicializálása
torch.backends.cudnn.deterministic = True  

# a neurális hálók tanításánál a véletlenszám-generálásnak nagy szerepe van
# érdemes a random seedet fixálni, hogy minden futtatásra ugyanazt az eredményt kapjuk
SEED = 202004
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Szövegosztályozás mély tanulással

A [5.](https://colab.research.google.com/drive/1iSIS8Z_iC8a4DjG4LE3MC04ZThcZPio-#scrollTo=SBHAiqDQoYfZ) órán megoldott szövegosztályozási feladatra fogunk adni itt egy mély gépi tanulási megoldást. Ugyanaz a feladat, véleményosztályozás. Ugyanazon az adatbázison, így az eredmények összehasonlíthatóak a klasszikus gépi tanulási eredményekkel.

## Szózsák alapú neurális hálózat

In [ ]:
import pandas as pd
train_data = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/train.tsv', sep='\t')
test_data  = pd.read_csv('https://github.com/rfarkas/student_data/raw/main/sentiment/test.tsv', sep='\t')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer()
cv_counts = vectorizer.fit_transform(train_data.text)
idf_transformer = TfidfTransformer(use_idf=True).fit(cv_counts)
features = idf_transformer.transform(cv_counts)
test_features = idf_transformer.transform(vectorizer.transform(test_data.text))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
model = SGDClassifier().fit(features, train_data.label)
accuracy_score(y_true=test_data.label, y_pred=model.predict(test_features))

0.7893333333333333

In [ ]:
### ritka mátrixot tensor formátumra alakítjuk
import numpy as np
X_train_tensor = torch.from_numpy(features.todense()).float()
X_test_tensor = torch.from_numpy(test_features.todense()).float()

In [ ]:
### PyTorch-ban még a célváltozó sem lehet diszkrét...
### A LabelEncoder véletlenszerűen Int-eket rendel az egyes értékekhez
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train_tensor = torch.as_tensor(le.fit_transform(train_data.label))
Y_test_tensor  = torch.as_tensor(le.transform(test_data.label)) 

In [ ]:
### Jellemzőtér (=bemeneti réteg) dimenziói és célváltozók száma (=kimeneti réteg dimenziója)
VOCAB_SIZE = len(vectorizer.vocabulary_)
OUT_CLASSES = 3

In [ ]:
### A legegyszerűbb neurális háló (ami megegyezik a lineáris géppel)
class LM_Network(torch.nn.Module):
     def __init__(self,vocab_size,out_classes):
        super().__init__()
        self.linear = torch.nn.Linear(vocab_size,out_classes)
     def forward(self,x):
        return self.linear(x)

model = LM_Network(VOCAB_SIZE,OUT_CLASSES)
print(model)

#predikció a random hálóval
model(X_train_tensor[1])

LM_Network(
  (linear): Linear(in_features=24285, out_features=3, bias=True)
)


tensor([-0.0019, -0.0008,  0.0132], grad_fn=<AddBackward0>)

In [ ]:
### 1 rejtett réteget tartalmazó neuárlis hálózat
class MLP_Network(torch.nn.Module):
  def __init__(self,vocab_size,hidden_units,num_classes): 
      super().__init__()
      #First fully connected layer
      self.fc1 = torch.nn.Linear(vocab_size,hidden_units)
      #Second fully connected layer
      self.fc2 = torch.nn.Linear(hidden_units,num_classes)
      #Final output of sigmoid function      
      self.output = torch.nn.Sigmoid()
  
  def forward(self,x):
      fc1 = self.fc1(x)
      fc2 = self.fc2(fc1)
      output = self.output(fc2)
      return output

HIDDEN_UNITS = 100
model = MLP_Network(VOCAB_SIZE, HIDDEN_UNITS, OUT_CLASSES)
print(model)

#Prediction without training
model(X_train_tensor[1])

MLP_Network(
  (fc1): Linear(in_features=24285, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=3, bias=True)
  (output): Sigmoid()
)


tensor([0.5247, 0.4864, 0.4760], grad_fn=<SigmoidBackward>)

In [ ]:
### Kiértékelő függvény 

def accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True) # a 3 osztályra adott kimeneti érték közül melyik a legnagyobb
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum(dtype=float) / y.shape[0]

In [ ]:
### ha az epoch végén egy független validációs halmazon is ki akarjuk értékelni a modellt:

def evaluate(model, iterator):
    epoch_acc = 0
    model.eval()  # inicializálás 
    with torch.no_grad():
        for batch in iterator:
            # predikció
            predictions = model(batch[0])
            # kiértékelés
            acc = accuracy(predictions, batch[1].long())
            epoch_acc += acc.item()
        
    return epoch_acc / len(iterator)

In [ ]:
from torch.utils.data import Dataset, TensorDataset
train_data = TensorDataset(X_train_tensor, Y_train_tensor)
test_data  = TensorDataset(X_test_tensor,  Y_test_tensor)

In [ ]:
### Ha egy adatbázison akarunk végigmenni akkor ahhoz iterátort kell definiálni
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

In [ ]:
# random hálózat kiértékelése az egész adatbázison
evaluate(model, train_loader)

0.33328609221466365

In [ ]:
### A tanítás során többször végigmegyünk a tanító adatbázison (egy kör egy epoch)
def train(model, iterator, optimizer, criterion):
    # minden epoch végén ellenőrízni fogjuk az accuracyt
    epoch_acc = 0
    
    model.train() # inicializálás
    for batch in iterator:
        # predikáljuk le a tanító példákat az aktuális paraméterekkel:
        optimizer.zero_grad()   
        predictions = model(batch[0])
                
        # a háló aktuális paraméterivel ennyi a hiba a batchen:
        loss = criterion(predictions, batch[1].long())
        acc = accuracy(predictions, batch[1].long())   
        
        # hibavisszaterjesztéssel (backpropagation) javítunk a paramétereken:
        loss.backward()
        optimizer.step()      
        
        epoch_acc += acc.item()    
      
    return epoch_acc / len(iterator)

In [ ]:
### Neurális hálózat tanítása
%%time
NUM_EPOCHS = 5
BATCH_SIZE = 64

#Neurális háló architektúra megadása
model = MLP_Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES)

#optimalizáló eljárás
import torch.optim as optim
optimizer = optim.Adam(model.parameters()) # ADAM optimalizáló algoritmus

#célfüggvény
import torch.nn as nn
loss_fun = nn.CrossEntropyLoss() 

iterator = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True)
for i in range(NUM_EPOCHS):
   print(i, ". epoch acc:", train(model, iterator, optimizer, loss_fun))

0 . epoch acc: 0.6318475758396533
1 . epoch acc: 0.8453864210906464
2 . epoch acc: 0.9025172670639221
3 . epoch acc: 0.9384677455760202
4 . epoch acc: 0.9621168517515349
CPU times: user 24.3 s, sys: 1.2 s, total: 25.5 s
Wall time: 25.4 s


In [ ]:
### Kiértékelés a teszt halmazon
test_loader = DataLoader(test_data,batch_size=16, shuffle=True)
evaluate(model, test_loader)

0.7799202127659575

In [ ]:
### Futtassunk mindent GPU-n!
%%time
NUM_EPOCHS = 5
BATCH_SIZE = 64

#Initialize model
model = MLP_Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES).to(device)

#Initialize optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters()) # ADAM optimalizáló algoritmus
import torch.nn as nn
loss_fun = nn.CrossEntropyLoss().to(device)

X_train_tensor = X_train_tensor.to(device)
Y_train_tensor = Y_train_tensor.to(device)
train_data = TensorDataset(X_train_tensor, Y_train_tensor)
iterator = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True)

for i in range(NUM_EPOCHS):
   print(i, ". epoch acc:", train(model, iterator, optimizer, loss_fun))

0 . epoch acc: 0.6946071460816179
1 . epoch acc: 0.8498894005055977
2 . epoch acc: 0.8993262459371614
3 . epoch acc: 0.9361795774647887
4 . epoch acc: 0.9607569293968942
CPU times: user 4.51 s, sys: 1.37 s, total: 5.88 s
Wall time: 17.2 s


In [ ]:
### mindent egyazon device-on kell futtatni...
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)
test_data = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_data,batch_size=16, shuffle=True)
evaluate(model, test_loader)

0.7768173758865248

## Szóbeágyazás alapú neurális hálók

In [ ]:
# torchtext egy szöveges adat feldolozására szolgáló segéd lib
from torchtext.legacy import data  

# a szöveges adat kezelésének módját lehet beállítani a torchtext-nek
TEXT  = data.Field(tokenize = 'spacy') # A SpaCy egy hasonló csomag, mint az NLTK, annak a tokenizálója van beépítve
# osztálycímke típusát állítjuk be:
LABEL = data.LabelField(dtype = torch.float) # neurális hálóknál minden float (nincsen diszkrét célváltozó)

In [ ]:
### Adatbetöltés

# Itt sajnos le kell töltenünk a fájlt
import urllib 
url = 'https://github.com/rfarkas/student_data/raw/main/sentiment/train.tsv'
urllib.request.urlretrieve(url,'train.tsv') # a temporálisan notebookhoz rendelt tárhelyre kerül a Google felhőjében

# Definiálnunk kell, hogy melyik oszlop milyen típusú
# első oszlop (ID) elhagyható, utána címke, végül a szöveg
fields = [(None, None), ('label', LABEL), ('text',TEXT)]

train_data = data.TabularDataset(path = 'train.tsv',
                                 fields = fields, # oszlopok értelmezése
                                 skip_header = True, # az első sorbeli oszlopneveket kihagyjuk. Az oszlopok neveit a fields-ben már megadtuk
                                 format = 'tsv')  # a tsv formátum megadja a sep='\t' is

In [ ]:
# az adatbázis betöltésekor lefutnak a szövegelőfeldolgozó lépések is (most csak a spacy tokenizálás)
print(vars(train_data.examples[0])) # examples[0] az első egyedünk
print('Tanító példák száma:', len(train_data.examples))

{'label': 'NEGATIVE', 'text': ['HSBC', "'", 'sorry', "'", 'for', 'aiding', 'Mexican', 'drugs', 'lords', ',', 'rogue', 'states', 'and', 'terrorists', 'http://gu.com/p/394tx/tw', '\xa0']}
Tanító példák száma: 9063


In [ ]:
### Kiértékelő adatbázis betöltése

url = 'https://raw.githubusercontent.com/rfarkas/student_data/main/sentiment/test.tsv'
urllib.request.urlretrieve(url,'test.tsv')
test_data = data.TabularDataset(path = 'test.tsv',
                                    fields = fields, # oszlopok értelmezése
                                    skip_header = True,
                                    format = 'tsv')  

A gyorsabb működés érdekében a deep learning keretrendszerek fix méretű mátrixokkal dolgoznak. Az egyik fix méret a **szótár** mérete. A ritka szavakat inkább eldobjuk. Általában egy előre beállított szótármérettel dolgozunk, a leggyakoribb N szó kerül felhasználásra. A ritka szavak - amik nem férnek bele az N elemű szótárba - lecserélődnek `<unk>` (unknown) tokenre. A szótárat csak a tanító adatbázisból építjük! Ezzel is szimuláljuk, hogy ismeretlen példákon lehetnek ismeretlen szavak.

A deep learning modellek ún. **szóbeágyazás**okat használnak tokenek leírására. Egy szóbeágyazás egy szóhoz egy numerikus vektort rendel. Ha két vektor közel van egymáshoz (pl. euklideszi vektortávolság szerint), akkor a két szó jelentése valamilyen értelemben hasonlít egymáshoz. Precízebben, két szóvektor akkor van közel egymáshoz ha hasonló mondatkörnyezetekben fordulnak elő. Egy fajta szóbeágyazás a [word2vec](https://towardsdatascience.com/understanding-word2vec-embedding-in-practice-3e9b8985953), de itt mi a [Glove](https://nlp.stanford.edu/projects/glove/) szóbeágyazást fogjuk használni, ami 6 millárd tokennyi szövegen ágyazta be az angol szavakat egy 100 dimenziós vektortérbe.

In [ ]:
MAX_VOCAB_SIZE = 15000 # szótár mérete
TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d") # ez először kitömörít 860MBot. A Google felhőjébe, nem lokálisan hozzád!
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [03:18, 4.34MB/s]                           
100%|█████████▉| 399999/400000 [00:27<00:00, 14430.94it/s]


In [ ]:
#print("Szótár mérete:",len(TEXT.vocab))
print("Osztálycímkék száma:",len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(10)) # leggyakorbb 10 szó és gyakoriságuk

Osztálycímkék száma: 3
[('#', 4001), (',', 3967), ('.', 3873), ('\xa0', 3660), ('the', 3339), (':', 3230), ('a', 3125), ('to', 2522), ('-', 2348), ('!', 2311)]


Szótár mérete 15002 mert a 15K szó mellé 2db technikai token is kerül, `<unk>` és `<pad>`. A szövegek hosszának is állandónak kell lennie a gyors mátrixműveletek érdekében. Ha a beállított fix szöveghossznál hosszabb szöveget adunk be, a hosszon túlllógó szavak elvesznek. Ha a fix hossznál rövidebb a szövegünk, akkor a hiányzó szöveget `<pad>` tokenekkel töltjük ki.

Például ha 5 a fix szöveghossz, az 'I hate you' mondat tokensorozata az lesz, hogy `['I', 'hate', 'you', '<pad>', '<pad>']`

In [ ]:
print(TEXT.vocab.itos[:10]) # szótár index szerinti első 10 eleme

['<unk>', '<pad>', '#', ',', '.', '\xa0', 'the', ':', 'a', 'to']


## Konvolúciós Neurális Hálózat tanítása

Egy ún **Konvulúciós Neurális Hálózatot** fogunk építani és tanítani a szövegosztályozási feladathoz (lásd [elméleti rész](TODO)). Egy bővebb PyTorch tutorial [itt](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb). 


In [ ]:
# a gyors elosztott számításokhoz egyszerre több példával dolgozunk (batchek)
BATCH_SIZE = 64

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

### Háló szerkezetének megadása

Minden feladatra saját hálózatot építhetünk az egyes neuron rétegek megadásával. Ehhez egy új osztályt kell definiálni, legalább konstruktorral és forward() metódussal.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, kernel_size, output_dim, pad_idx):
        super().__init__()
        
        # legalul a szóbeágyazások vannak, itt minden szót egy embedding_dim dimenziós vektor ír le, amit a Glove szótárból olvaunk ki
        # <pad> speciálisan kezelődik (arra nincs értelme szóbeágyazás vektort használni) ezért meg kell adni az indexét
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # utána jön a konvolúciós réteg (rétegek), itt a kernel mérete az "ablakméret"
        self.conv = nn.Conv1d(in_channels = embedding_dim, 
                              out_channels = n_filters, 
                              kernel_size = kernel_size)
        
        # végül a kimeneti réteg, ami egy egyszerű lineáris réteg
        self.fc = nn.Linear(n_filters, output_dim)
                        
    # amikor a szöveget előrefelé ("alulról felfelé" a rétegeken át) feldolgozza a háló
    def forward(self, text):
        
        # a bemenet a batch_size darab szöveg, mindegyik pontosan sent_len hosszúságú
        # text = [batch size, sent len]
        text = text.permute(1, 0) # megcseréljük a dimenziókat
        # text = [sent len, batch size]
 
        # kiolvassuk a Glove szótárból a szóbeágyazási vektorokat
        embedded = self.embedding(text)
        # ez már egy 3 dimenziós tömb (tenzor):
        # embedded = [batch size, sent len, emb dim]
        embedded = embedded.permute(0, 2, 1)
        #embedded = [batch size, emb dim, sent len]
        
        # ezután a konvolúciós réteg a RelU aktivációs függvényt használja
        conved = F.relu(self.conv(embedded))
            
        # ennek a tenzornak a méretei:
        # conved = [batch size, n_filters, sent len - filter_size + 1]
        
        # tovább tömörítjük: 
        pooled = F.max_pool1d(conved, conved.shape[2]).squeeze(2)
        # pooled = [batch size, n_filters]

        # a háló kimenetét a lineáris réteg számolja ki                    
        return self.fc(pooled)

In [ ]:
### a háló példányosítása

size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
n_filters = 100
kernel_size = 3
output_dim = len(LABEL.vocab)
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(size_of_vocab, embedding_dim, n_filters, kernel_size, output_dim, pad_idx)                  

In [ ]:
# a háló rétegei:
print(model)

# összesen ennyi paramétert kell tanítanunk:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model), "tanulandó változó") 
# ahhoz, hogy 1.5M válozót beállítsunk=megtanuljunk nagyon sok tanító példa kell(ene)!

CNN(
  (embedding): Embedding(15002, 100, padding_idx=1)
  (conv): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
1530603 tanulandó változó


In [ ]:
### inicializáljuk a szóbeágyazási réteget
### itt a Glove szótárból csak azoknak a szavaknak a vektorát olvassuk be, amire szükség van (a mi szótárunk)
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

torch.Size([15002, 100])


### CNN tanítása

A neurális hálók tanítása egy optimalizációs feladat megoldásával törénik. Úgy akrjuk beállítani az 1.5M változót, hogy minimalizáljuk a háló kimenete és a tényleg osztálycímke közti eltérést.

In [ ]:
### A tanítás során többször végigmegyünk a tanító adatbázison (egy kör egy epoch)

def train(model, iterator, optimizer, criterion):
    # minden epoch végén ellenőrízni fogjuk az accuracyt
    epoch_acc = 0
    
    model.train() # inicializálás
    for batch in iterator:
        # predikáljuk le a tanító példákat az aktuális paraméterekkel:
        optimizer.zero_grad()   
        predictions = model(batch.text)
                
        # a háló aktuális paraméterivel ennyi a hiba a batchen:
        loss = criterion(predictions, batch.label.long())
        acc = accuracy(predictions, batch.label)   
        
        # hibavisszaterjesztéssel (backpropagation) javítunk a paramétereken:
        loss.backward()
        optimizer.step()      
        
        epoch_acc += acc.item()    
      
    return epoch_acc / len(iterator)

In [ ]:
# GPU-n akarunk tanítani:
model = CNN(size_of_vocab, embedding_dim, n_filters, kernel_size, output_dim, pad_idx)   
model = model.to(device)

import torch.optim as optim
optimizer = optim.Adam(model.parameters()) # ADAM optimalizáló algoritmus
criterion = nn.CrossEntropyLoss() # hibafüggvény többosztályos feladatokhoz
criterion = criterion.to(device)

In [ ]:
%%time
### mehet a tanítás!

NUM_EPOCHS = 5

for i in range(NUM_EPOCHS):
    print(i, ". epoch acc:", train(model, train_iterator, optimizer, criterion))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 . epoch acc: 0.5599409759841097
1 . epoch acc: 0.7653372156013001
2 . epoch acc: 0.8459591684723726
3 . epoch acc: 0.9043257945106536
4 . epoch acc: 0.9483624503430842
CPU times: user 5.05 s, sys: 88.6 ms, total: 5.14 s
Wall time: 5.35 s


### CNN kiértékelése kiértékelő halmazon

In [ ]:
### ha az epoch végén egy független validációs halmazon is ki akarjuk értékelni a modellt:

def evaluate(model, iterator):
    epoch_acc = 0
    model.eval()  # inicializálás 
    with torch.no_grad():
        for batch in iterator:
            # predikció
            predictions = model(batch.text)
            # kiértékelés
            acc = accuracy(predictions, batch.label.long())
            epoch_acc += acc.item()
        
    return epoch_acc / len(iterator)

In [ ]:
evaluate(model, test_iterator)

AttributeError: ignored

Mivel a tanító és kiértékelő adatbázisok valamint kiértékelési metrika megegyezzik az [5. órai](TODO) "klasszikus" gépi tanulási kísérletben használttal (79% accuracy), ezért az eredmények közvetlenül összehasonlíthatóak. Megállapíthatjuk, hogy ekkora tanító adatbázison a deep learning, még a szóbeágyazásokkal sem tud jobb eredményt elérni, mint 
a szózsák alapú lineáris gép.

# Gyakorló fealdatok

Futtasd az órai notebook-ot, hajtsd végre az alábbi módosításokat a rendszeren! 

1. Próbálj ki egy másik szóbeágyazást, úgy mennyi lesz a kiértékelő halmazon a pontosság?

     Elérhető szóbeágyazásokhoz lásd [load_vector fgv leírása](https://torchtext.readthedocs.io/en/latest/vocab.html#vocab). Vigyázz, az embedding_dim változót is be kell állítani a használt beágyazás dimenziójára!

2. Ha a konvolúció ablakméretét 5-re állítjuk, mennyi lesz a kiértékelő halmazon a pontosság?